*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split

import seaborn as sns

In [ ]:
!pip install -q sklearn

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
#
#dividir en test y train
p_train = 0.80 # Porcentaje de train.

dataset['is_train'] = np.random.uniform(0, 1, len(dataset)) <= p_train
train_dataset, test_dataset = dataset[dataset['is_train']==True], dataset[dataset['is_train']==False]
dataset = dataset.drop('is_train', 1)

print("Ejemplos usados para entrenar: ", len(train_dataset))
print("Ejemplos usados para test: ", len(test_dataset))

In [ ]:
#crear los labels
train_labels = train_dataset.pop("expenses")
test_labels = test_dataset.pop("expenses")
test_labels.head()

In [ ]:
#volver numéricas todas las variables
test_dataset = pd.get_dummies(test_dataset, columns = ["sex","smoker","region","is_train"], drop_first=True)
train_dataset = pd.get_dummies(train_dataset, columns = ["sex","smoker","region"], drop_first=True)

In [ ]:
test_dataset["is_train"] = test_dataset["is_train"].map({"True":1,"False":0})
train_dataset["is_train"] = train_dataset["is_train"].map({"True":1,"False":0})

In [ ]:
CATEGORICAL_COLUMNS = []
NUMERIC_COLUMNS = ['age', 'bmi',"children",'sex', 'smoker', "region","is_train"]

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = train_dataset[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

In [ ]:
#dataframe = dataset


In [ ]:
#test_labels.head()

In [ ]:
#train_dataset['region'].dtype
#train_labels = train_labels(range(0,train_labels))

In [ ]:
#funcion input
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(train_dataset, train_labels)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(test_dataset, test_labels, num_epochs=1, shuffle=False)

In [ ]:
len(feature_columns)

In [ ]:
#creando el modelo
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns, n_classes=6)
# We create a linear estimtor by passing the feature columns we created earlier

In [ ]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_dataset))

In [ ]:
#entrenando el modelo
linear_est.train(train_input_fn)  # train
result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on tetsing data

clear_output()  # clears console output
print(result['accuracy'])  # the result variable is simply a dict of stats about our model

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
